# Analisador Audio Texto

Caderno para conversão e análise de audio.

Pré requisitos:

Instalar o ffmpeg para conversão de formatos de áudio.

Pode-se instalar pelo Homebrew por meio do comando: brew install ffmpeg.

Instalar a biblioteca vosk: pip install vosk

Baixar o modelo do vosk em português no site:
https://alphacephei.com/vosk/models
Arquivo: vosk-model-small-pt-0.3


In [126]:
# Importação das bibliotecas
import os

import subprocess
import sys

from vosk import Model, KaldiRecognizer, SetLogLevel

import json

import pandas as pd

import IPython

In [45]:
# Ler vários formatos de arquivo de áudio de uma pasta.
pasta = os.getcwd() + r'/Audio' + r'/'
lista_arquivos = os.listdir(pasta)
print(f"Quantidade de aquivos na pasta {pasta}: ", len(lista_arquivos))
print(lista_arquivos)

Quantidade de aquivos na pasta /Users/andreluiz/Documents/projetos/analisador-audio-texto/Audio/:  8
['AUD-20190701-WA0005.wav', 'AUD-20200128-WA0006.mp3', 'Voz 001.m4a', 'PTT-20200304-WA0005.opus', 'PTT-20200304-WA0006.opus', 'AUD-20190701-WA0005.mp3', 'AUD-20200216-WA0011.aac', 'AUD-20190723-WA0001.m4a']


In [29]:
pasta

'/Users/andreluiz/Documents/projetos/analisador-audio-texto/Audio/'

## Transcrever os áudios e gravar em um json

In [87]:
# Inicializar variáveis e modelo

SAMPLE_RATE = 16000

SetLogLevel(0)

model = Model('models/modelos/vosk-model-small-pt-0.3')
rec = KaldiRecognizer(model, SAMPLE_RATE)
lista_transcricoes = []

# Ler cada arquivo e transcrever
for arquivo in lista_arquivos:    
    with subprocess.Popen(["ffmpeg", "-loglevel", "quiet", "-i",
                            pasta + arquivo,
                            "-ar", str(SAMPLE_RATE) , "-ac", "1", "-f", "s16le", "-"],
                            stdout=subprocess.PIPE) as process:
        transcricao = ' '
        dic_transcricao = {}
  

        while True:
            data = process.stdout.read(4000)
            if len(data) == 0:
                break
            if rec.AcceptWaveform(data):
                dic_transcricao = json.loads(rec.Result())
                transcricao = transcricao + ' ' + dic_transcricao['text']

        dic_transcricao = json.loads(rec.FinalResult())
        transcricao = transcricao + ' ' + dic_transcricao['text']
        lista_transcricoes.append(transcricao)

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from models/modelos/vosk-model-small-pt-0.3/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from models/modelos/vosk-model-small-pt-0.3/HCLr.fst models/modelos/vosk-model-small-pt-0.3/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo models/modelos/vosk-model-small-pt-0.3/word_boundary.int


In [88]:
len(lista_transcricoes)

8

In [99]:
print(lista_arquivos[1])
print(lista_transcricoes[1])

AUD-20200128-WA0006.mp3
  uma o  gostou o coração de quem se entrega à oração tem mil histórias para contar costuma fazer bem o coração de quem conversa com são muita coisa hoje a terça feira dia vinte e oito de janeiro e queremos fazer memória de santo tomás de aquino este grande doutor da igreja homem de profunda sabedoria seja dos seus escritos de filosofia ou de teologia santo tomás de aquino que foi dominicano compromisso sado pela busca da verdade que interceda por nossa nesta terça feira em nossa caminhada de fé e que saibamos nós viver entre fé e a nossa razão de forma equilibrada para subirmos até deus nosso salvador iniciamos em nome do pai e do filho e do espírito santo amém que a graça de nosso senhor jesus cristo o amor do pai e a comunhão do espírito santo estejam sempre convosco bendito seja deus que não se reuniu no amor de cristo unidos em amor de cristo ou com atenção as palavras do santo evangelho não importa não  numa casa evangelho de jesus cristo segundo marcos ca

In [152]:
# Cria um DataFrame com as transcrições
# O nome do arquivo é o índice do Dataframe
df_audio_transcricoes =  pd.DataFrame(lista_transcricoes, index=lista_arquivos, columns=['transcricao'])
df_audio_transcricoes

,transcricao
AUD-20190701-WA0005.wav,o costuma fazer o coração de quem se entre...
AUD-20200128-WA0006.mp3,uma o gostou o coração de quem se entrega à...
Voz 001.m4a,o serviço depois só de comer e os de e...
PTT-20200304-WA0005.opus,pa já sabe estou saindo pra ir pra lá o lá p...
PTT-20200304-WA0006.opus,mensagem eu tenho longo de dois chega lá voc...
AUD-20190701-WA0005.mp3,o gostou o coração de quem se entrega à or...
AUD-20200216-WA0011.aac,gostou o coração de quem se entrega à ora...
AUD-20190723-WA0001.m4a,agora eu moral podem decidir qual é a omissã...


In [153]:
# Realizar filtros pelo índice com nome do arquivo
arquivo_selecionado = 'AUD-20200216-WA0011.aac'

df_audio_transcricoes['transcricao'].loc[arquivo_selecionado]

#IPython.display.Audio(pasta + arquivo_selecionado)



'     gostou o coração de quem se entrega à oração mil histórias para contar costuma bem o coração de quem conversa com muita coisa neste domingo dia dezesseis de fevereiro estamos vivendo dia do senhor é o sexto domingo do tempo comum que nós saibamos deixar deus está acima de todas as coisas neste dia nos coloquemos em oração seja neste momento seja também no momento da santa missa iniciemos em nome do pai e do filho e do espírito santo amém que a graça de nosso senhor jesus cristo o amor do pai e a comunhão do espírito santo estejam sempre convosco bendito seja deus que não se reuniu no amor de cristo deixemos que as luzes de deus cheguem a nossa vida através das palavras do santo evangelho não importa se não  o evangelho de jesus cristo segundo mateus capítulo cinco de ciclos de vinte a vinte e dois vinte e sete vinte e oito trinta e três a trinta e quatro naquele tempo disse jesus a seus discípulos eu vos digo se a vossa justiça não for maior que a justiça dos mestres da lei e dos

In [167]:
# Realiar filtro por palavras-chaves em todos as transcrições
lista_palavras_chaves = ['higienização', 'família']
palavra_chave = '|'.join(lista_palavras_chaves)

df_audio_transcricoes['transcricao'].str.contains(palavra_chave)
#df_audio_transcricoes[df_audio_transcricoes['transcricao'].str.contains(palavra_chave)]


# Podemos mostrar tanto os audios em que APARECEM (True) as palavras chaves como NAO APARECEM (False)
# Filtra as colunas que NÃO APARECE as palavras chave. 
#df_audio_transcricoes.query(f"{serie_resultado} == True " )

AUD-20190701-WA0005.wav      True
AUD-20200128-WA0006.mp3      True
Voz 001.m4a                 False
PTT-20200304-WA0005.opus    False
PTT-20200304-WA0006.opus    False
AUD-20190701-WA0005.mp3      True
AUD-20200216-WA0011.aac      True
AUD-20190723-WA0001.m4a     False
Name: transcricao, dtype: bool